## Data cleaning

Before any analysis and exploration of the dataset can be accomplished, I always perform data cleaning. This is a crucial step. Clean data will result in more accuracy.

The data cleaning process will occur in five different parts:
1. Missing Data
2. Redundant Columns
3. Time Data
4. Datetime Extraction
5. Misc. Data

At the end of this I'll create a new csv file that is cleaned, and will be ready for exploration and analysis.

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
import missingno as msno
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [2]:
# Load the csv file
csv = pd.read_csv("bicycle_sharing.csv", low_memory=False)
csv.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Year
0,A847FADBBC638E45,docked_bike,2020-04-26 17:45:14,2020-04-26 18:12:03,Eckhart Park,86,Lincoln Ave & Diversey Pkwy,152.0,41.90,-87.66,41.93,-87.66,member,2020
1,5405B80E996FF60D,docked_bike,2020-04-17 17:08:54,2020-04-17 17:17:03,Drake Ave & Fullerton Ave,503,Kosciuszko Park,499.0,41.92,-87.72,41.93,-87.72,member,2020
2,5DD24A79A4E006F4,docked_bike,2020-04-01 17:54:13,2020-04-01 18:08:36,McClurg Ct & Erie St,142,Indiana Ave & Roosevelt Rd,255.0,41.89,-87.62,41.87,-87.62,member,2020
3,2A59BBDF5CDBA725,docked_bike,2020-04-07 12:50:19,2020-04-07 13:02:31,California Ave & Division St,216,Wood St & Augusta Blvd,657.0,41.90,-87.70,41.90,-87.67,member,2020
4,27AD306C119C6158,docked_bike,2020-04-18 10:22:59,2020-04-18 11:15:54,Rush St & Hubbard St,125,Sheridan Rd & Lawrence Ave,323.0,41.89,-87.63,41.97,-87.65,casual,2020


In [3]:
csv.info(memory_usage="deep")
### Memory usage is very high

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14758322 entries, 0 to 14758321
Data columns (total 14 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
 13  Year                int64  
dtypes: float64(4), int64(1), object(9)
memory usage: 9.2 GB


## 1. Missing data
- Missing data for station name might indicate that the rides started and/or ended at a location that was not a station
- is station id important? Probably not

In [4]:
# Percent of missing data, sorted descending order
(csv.apply(pd.isnull).sum()/csv.shape[0] * 100).sort_values(ascending=False)

end_station_id       12.18
end_station_name     12.18
start_station_id     11.33
start_station_name   11.32
end_lat               0.10
end_lng               0.10
ride_id               0.00
rideable_type         0.00
started_at            0.00
ended_at              0.00
start_lat             0.00
start_lng             0.00
member_casual         0.00
Year                  0.00
dtype: float64

In [5]:
# Filter the dataframe for rows with any missing data
null_data = csv[csv.isnull().any(axis=1)]

In [6]:
# Gives us a glimpse of some of the missing rows using the sample function
null_data.sample(20)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Year
9315098,737781D50963674F,electric_bike,2022-04-14 15:38:59,2022-04-14 15:40:27,NaN,NaN,NaN,NaN,41.80,-87.60,41.80,-87.60,member,2022
13347208,E10E91DB65A47025,electric_bike,2022-10-15 15:53:58,2022-10-15 15:57:29,NaN,NaN,Sheffield Ave & Webster Ave,TA1309000033,41.93,-87.66,41.92,-87.65,member,2022
10889894,9BD6D03E20997717,electric_bike,2022-06-29 10:08:35,2022-06-29 10:31:20,NaN,NaN,Paulina St & Flournoy St,KA1504000104,41.91,-87.67,41.87,-87.67,member,2022
13008846,32E15E9EEA171A6B,electric_bike,2022-09-22 15:26:56,2022-09-22 15:30:53,State St & 33rd St,13216,NaN,NaN,41.83,-87.63,41.84,-87.62,member,2022
8683363,BAA0F601EEE9FCBD,electric_bike,2021-12-14 16:27:33,2021-12-14 16:36:32,NaN,NaN,NaN,NaN,41.93,-87.70,41.92,-87.69,casual,2021
13479328,6F372716504B3EF8,electric_bike,2022-10-19 10:41:34,2022-10-19 10:51:54,NaN,NaN,Clarendon Ave & Junior Ter,13389,41.94,-87.64,41.96,-87.65,member,2022
5048991,1DBEFDC437AD8E7D,electric_bike,2021-06-05 20:36:51,2021-06-05 21:03:02,NaN,NaN,Damen Ave & Cullerton St,13089,41.84,-87.71,41.85,-87.68,casual,2021
6559342,3894EA230E0CB894,electric_bike,2021-08-08 14:02:29,2021-08-08 14:08:39,NaN,NaN,NaN,NaN,41.94,-87.64,41.94,-87.64,member,2021
13017576,1F8B239CB8D221AE,electric_bike,2022-09-24 17:04:00,2022-09-24 17:30:38,Christiana Ave & Lawrence Ave,15615,NaN,NaN,41.97,-87.71,41.94,-87.76,casual,2022
1196174,B82A12502BF6EC0F,electric_bike,2020-08-23 15:10:42,2020-08-23 16:18:41,NaN,NaN,Sheridan Rd & Lawrence Ave,323.0,41.96,-87.65,41.97,-87.65,casual,2020


### Ride IDs
- Ride ID column has a few duplicate rows
- I'm not entirely sure why the Ride ID column has duplicated rows, but they exist.
- This column should contain entirely unique IDs for each row, as this is a separate occurence of every ride (like a receipt for a purchase)

In [7]:
# Compare the length of the entire dataset with the number of unique ride IDs 
# This value will return false
# Indicates duplicate? 
len(csv["ride_id"]) == csv["ride_id"].nunique()

False

In [8]:
# Number of unique ride IDs
# Ride IDs are essentially transaction numbers that should be ALL different
csv["ride_id"].nunique()

14758113

In [9]:
# Drop the duplicated rows under the Ride ID column
csv[csv.duplicated("ride_id", keep=False) == True]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Year
2753661,8758E473B457691C,docked_bike,2020-11-25 08:54:55,2020-11-25 09:00:16,Wells St & Polk St,175.0,LaSalle St & Washington St,98.0,41.87,-87.63,41.88,-87.63,member,2020
2755411,47E0C01E8F7BD830,docked_bike,2020-11-25 17:38:30,2020-11-25 18:06:03,Ellis Ave & 60th St,426.0,Lake Park Ave & 56th St,345.0,41.79,-87.60,41.79,-87.59,member,2020
2771017,4AE7C88494448250,docked_bike,2020-11-25 09:19:07,2020-11-25 09:27:26,Paulina St & 18th St,205.0,Wolcott Ave & Polk St,342.0,41.86,-87.67,41.87,-87.67,member,2020
2771185,461DB5E36ADBF190,docked_bike,2020-11-25 15:33:21,2020-11-25 15:35:07,Clinton St & Lake St,66.0,Clinton St & Lake St,66.0,41.89,-87.64,41.89,-87.64,casual,2020
2772563,203E4F607EB792FC,docked_bike,2020-11-25 19:26:30,2020-11-25 19:32:58,Clark St & Chicago Ave,337.0,Larrabee St & Kingsbury St,48.0,41.90,-87.63,41.90,-87.64,member,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3111833,6C1279812BD9D0C2,docked_bike,2020-12-15 12:08:00,2020-11-26 08:25:29,Woodlawn Ave & 55th St,TA1307000164,Blackstone Ave & Hyde Park Blvd,13398,41.80,-87.60,41.80,-87.59,casual,2020
3112006,5EE680AA8D46F8B8,docked_bike,2020-12-15 12:14:03,2020-11-25 09:23:46,Calumet Ave & 18th St,13102,Wabash Ave & 9th St,TA1309000010,41.86,-87.62,41.87,-87.63,member,2020
3112867,286E58C4B46877DE,docked_bike,2020-12-15 11:55:29,2020-11-25 16:16:51,Honore St & Division St,TA1305000034,Damen Ave & Clybourn Ave,13271,41.90,-87.67,41.93,-87.68,member,2020
3113600,7074FF42B83EC6AD,docked_bike,2020-12-15 11:50:14,2020-11-25 13:42:44,Peoria St & Jackson Blvd,13158,Clinton St & Madison St,TA1305000032,41.88,-87.65,41.88,-87.64,member,2020


In [10]:
csv[csv["ride_id"].duplicated(keep=False)]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Year
2753661,8758E473B457691C,docked_bike,2020-11-25 08:54:55,2020-11-25 09:00:16,Wells St & Polk St,175.0,LaSalle St & Washington St,98.0,41.87,-87.63,41.88,-87.63,member,2020
2755411,47E0C01E8F7BD830,docked_bike,2020-11-25 17:38:30,2020-11-25 18:06:03,Ellis Ave & 60th St,426.0,Lake Park Ave & 56th St,345.0,41.79,-87.60,41.79,-87.59,member,2020
2771017,4AE7C88494448250,docked_bike,2020-11-25 09:19:07,2020-11-25 09:27:26,Paulina St & 18th St,205.0,Wolcott Ave & Polk St,342.0,41.86,-87.67,41.87,-87.67,member,2020
2771185,461DB5E36ADBF190,docked_bike,2020-11-25 15:33:21,2020-11-25 15:35:07,Clinton St & Lake St,66.0,Clinton St & Lake St,66.0,41.89,-87.64,41.89,-87.64,casual,2020
2772563,203E4F607EB792FC,docked_bike,2020-11-25 19:26:30,2020-11-25 19:32:58,Clark St & Chicago Ave,337.0,Larrabee St & Kingsbury St,48.0,41.90,-87.63,41.90,-87.64,member,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3111833,6C1279812BD9D0C2,docked_bike,2020-12-15 12:08:00,2020-11-26 08:25:29,Woodlawn Ave & 55th St,TA1307000164,Blackstone Ave & Hyde Park Blvd,13398,41.80,-87.60,41.80,-87.59,casual,2020
3112006,5EE680AA8D46F8B8,docked_bike,2020-12-15 12:14:03,2020-11-25 09:23:46,Calumet Ave & 18th St,13102,Wabash Ave & 9th St,TA1309000010,41.86,-87.62,41.87,-87.63,member,2020
3112867,286E58C4B46877DE,docked_bike,2020-12-15 11:55:29,2020-11-25 16:16:51,Honore St & Division St,TA1305000034,Damen Ave & Clybourn Ave,13271,41.90,-87.67,41.93,-87.68,member,2020
3113600,7074FF42B83EC6AD,docked_bike,2020-12-15 11:50:14,2020-11-25 13:42:44,Peoria St & Jackson Blvd,13158,Clinton St & Madison St,TA1305000032,41.88,-87.65,41.88,-87.64,member,2020


For some reason all the duplicated rows have a second "ride_id" that has a start date of *December 15, 2020* and an end date of *November 25, 2020*.
I've checked multiple "ride_id"s and it is the same way for all of them. As the "ride_id" is supposed to be a unique transaction ID I can only assume there's some kind of error in the data itself.
I am confident that the best solution is to get rid of the duplicated rows for a more accurate analysis. 

In [11]:
csv[csv["ride_id"] == "47E0C01E8F7BD830"]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Year
2755411,47E0C01E8F7BD830,docked_bike,2020-11-25 17:38:30,2020-11-25 18:06:03,Ellis Ave & 60th St,426.0,Lake Park Ave & 56th St,345.0,41.79,-87.60,41.79,-87.59,member,2020
3110497,47E0C01E8F7BD830,docked_bike,2020-12-15 12:00:02,2020-11-25 18:06:03,Ellis Ave & 60th St,KA1503000014,Lake Park Ave & 56th St,TA1309000063,41.79,-87.60,41.79,-87.59,member,2020


In [12]:
# Drop duplicate entries on the "ride_id" column
csv = csv.drop_duplicates(subset="ride_id",keep="first")

## 2. Drop unneccesary columns
Now that the duplicated rows based on the "ride_id" have been dealt with I'm going to remove the following columns:

- "ride_id": provides no relevant information

- start_station_id: **station name** is enough information

- end_station_id: **station name** is enough information

- I don't need the coordinates for the rides, I already plotted some of the coordinates on Tableau, and it's a complete mess. All of these rides take place in Chicago, Illinois. Plotting the locations will not serve us in this analysis.

In [13]:
### Drop columns
cols_to_drop =[
    "ride_id",
    "start_station_id",
    "end_station_id",
    "start_lat",
    "start_lng",
    "end_lat",
    "end_lng"
]

# Reassign to a new dataframe
new_df = csv.drop(cols_to_drop, axis = 1)
new_df

,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,Year
0,docked_bike,2020-04-26 17:45:14,2020-04-26 18:12:03,Eckhart Park,Lincoln Ave & Diversey Pkwy,member,2020
1,docked_bike,2020-04-17 17:08:54,2020-04-17 17:17:03,Drake Ave & Fullerton Ave,Kosciuszko Park,member,2020
2,docked_bike,2020-04-01 17:54:13,2020-04-01 18:08:36,McClurg Ct & Erie St,Indiana Ave & Roosevelt Rd,member,2020
3,docked_bike,2020-04-07 12:50:19,2020-04-07 13:02:31,California Ave & Division St,Wood St & Augusta Blvd,member,2020
4,docked_bike,2020-04-18 10:22:59,2020-04-18 11:15:54,Rush St & Hubbard St,Sheridan Rd & Lawrence Ave,casual,2020
...,...,...,...,...,...,...,...
14758317,classic_bike,2023-02-08 21:57:22,2023-02-08 22:08:06,Clark St & Wrightwood Ave,Sheffield Ave & Waveland Ave,member,2023
14758318,electric_bike,2023-02-19 11:29:09,2023-02-19 11:39:11,Ogden Ave & Roosevelt Rd,Delano Ct & Roosevelt Rd,member,2023
14758319,electric_bike,2023-02-07 09:01:33,2023-02-07 09:16:53,Clark St & Wrightwood Ave,Canal St & Madison St,casual,2023
14758320,electric_bike,2023-02-22 08:33:22,2023-02-22 08:50:11,Clark St & Wrightwood Ave,Canal St & Madison St,casual,2023


In [14]:
### Check current memory ussage
new_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14758113 entries, 0 to 14758321
Data columns (total 7 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   rideable_type       object
 1   started_at          object
 2   ended_at            object
 3   start_station_name  object
 4   end_station_name    object
 5   member_casual       object
 6   Year                int64 
dtypes: int64(1), object(6)
memory usage: 6.2 GB


## 3. Dealing with time data
Pandas has set the "started_at" and "ended_at" columns to an object type, rather than a date time object.
I'll need to change some of these columns to a date time object for lower memory usage and usability.

In [15]:
new_df["started_at"] = pd.to_datetime(new_df["started_at"],format = "%Y-%m-%d %H:%M:%S")
new_df["ended_at"] = pd.to_datetime(new_df["ended_at"],format = "%Y-%m-%d %H:%M:%S")

In [16]:
### Check current memory ussage
new_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14758113 entries, 0 to 14758321
Data columns (total 7 columns):
 #   Column              Dtype         
---  ------              -----         
 0   rideable_type       object        
 1   started_at          datetime64[ns]
 2   ended_at            datetime64[ns]
 3   start_station_name  object        
 4   end_station_name    object        
 5   member_casual       object        
 6   Year                int64         
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 4.3 GB


In [17]:
### Percentage of missing values in the dataset
(new_df.apply(pd.isnull).sum()/new_df.shape[0] * 100).sort_values(ascending=False)

end_station_name     12.18
start_station_name   11.32
rideable_type         0.00
started_at            0.00
ended_at              0.00
member_casual         0.00
Year                  0.00
dtype: float64

## 4. Extract date time columns for analysis
I want to extract the specific months, days, and hours from the original "started_at" and "ended_at" columns for future use.
This task can be accomplished by using the pandas `.dt` function to extract the specific components that I desire.

I already have a column called "Year" that includes only the year. I performed this task earlier.

In [18]:
### Extract just the date from these columns
new_df["start_date"] = new_df["started_at"].dt.date
new_df["end_date"] = new_df["ended_at"].dt.date

### Extract the time 
new_df["start_time"] = new_df["started_at"].dt.time
new_df["end_time"] = new_df["ended_at"].dt.time

### Extract the month
new_df["start_month"] = new_df["started_at"].dt.month
new_df["end_month"] = new_df["ended_at"].dt.month

### Extract the day
new_df["start_day"] = new_df["started_at"].dt.day
new_df["end_day"] = new_df["ended_at"].dt.day


In [19]:
new_df

,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,Year,start_date,end_date,start_time,end_time,start_month,end_month,start_day,end_day
0,docked_bike,2020-04-26 17:45:14,2020-04-26 18:12:03,Eckhart Park,Lincoln Ave & Diversey Pkwy,member,2020,2020-04-26,2020-04-26,17:45:14,18:12:03,4,4,26,26
1,docked_bike,2020-04-17 17:08:54,2020-04-17 17:17:03,Drake Ave & Fullerton Ave,Kosciuszko Park,member,2020,2020-04-17,2020-04-17,17:08:54,17:17:03,4,4,17,17
2,docked_bike,2020-04-01 17:54:13,2020-04-01 18:08:36,McClurg Ct & Erie St,Indiana Ave & Roosevelt Rd,member,2020,2020-04-01,2020-04-01,17:54:13,18:08:36,4,4,1,1
3,docked_bike,2020-04-07 12:50:19,2020-04-07 13:02:31,California Ave & Division St,Wood St & Augusta Blvd,member,2020,2020-04-07,2020-04-07,12:50:19,13:02:31,4,4,7,7
4,docked_bike,2020-04-18 10:22:59,2020-04-18 11:15:54,Rush St & Hubbard St,Sheridan Rd & Lawrence Ave,casual,2020,2020-04-18,2020-04-18,10:22:59,11:15:54,4,4,18,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14758317,classic_bike,2023-02-08 21:57:22,2023-02-08 22:08:06,Clark St & Wrightwood Ave,Sheffield Ave & Waveland Ave,member,2023,2023-02-08,2023-02-08,21:57:22,22:08:06,2,2,8,8
14758318,electric_bike,2023-02-19 11:29:09,2023-02-19 11:39:11,Ogden Ave & Roosevelt Rd,Delano Ct & Roosevelt Rd,member,2023,2023-02-19,2023-02-19,11:29:09,11:39:11,2,2,19,19
14758319,electric_bike,2023-02-07 09:01:33,2023-02-07 09:16:53,Clark St & Wrightwood Ave,Canal St & Madison St,casual,2023,2023-02-07,2023-02-07,09:01:33,09:16:53,2,2,7,7
14758320,electric_bike,2023-02-22 08:33:22,2023-02-22 08:50:11,Clark St & Wrightwood Ave,Canal St & Madison St,casual,2023,2023-02-22,2023-02-22,08:33:22,08:50:11,2,2,22,22


In [20]:
### Check current memory ussage
new_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14758113 entries, 0 to 14758321
Data columns (total 15 columns):
 #   Column              Dtype         
---  ------              -----         
 0   rideable_type       object        
 1   started_at          datetime64[ns]
 2   ended_at            datetime64[ns]
 3   start_station_name  object        
 4   end_station_name    object        
 5   member_casual       object        
 6   Year                int64         
 7   start_date          object        
 8   end_date            object        
 9   start_time          object        
 10  end_time            object        
 11  start_month         int64         
 12  end_month           int64         
 13  start_day           int64         
 14  end_day             int64         
dtypes: datetime64[ns](2), int64(5), object(8)
memory usage: 7.2 GB


In [22]:
new_df.head()

,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,Year,start_date,end_date,start_time,end_time,start_month,end_month,start_day,end_day
0,docked_bike,2020-04-26 17:45:14,2020-04-26 18:12:03,Eckhart Park,Lincoln Ave & Diversey Pkwy,member,2020,2020-04-26,2020-04-26,17:45:14,18:12:03,4,4,26,26
1,docked_bike,2020-04-17 17:08:54,2020-04-17 17:17:03,Drake Ave & Fullerton Ave,Kosciuszko Park,member,2020,2020-04-17,2020-04-17,17:08:54,17:17:03,4,4,17,17
2,docked_bike,2020-04-01 17:54:13,2020-04-01 18:08:36,McClurg Ct & Erie St,Indiana Ave & Roosevelt Rd,member,2020,2020-04-01,2020-04-01,17:54:13,18:08:36,4,4,1,1
3,docked_bike,2020-04-07 12:50:19,2020-04-07 13:02:31,California Ave & Division St,Wood St & Augusta Blvd,member,2020,2020-04-07,2020-04-07,12:50:19,13:02:31,4,4,7,7
4,docked_bike,2020-04-18 10:22:59,2020-04-18 11:15:54,Rush St & Hubbard St,Sheridan Rd & Lawrence Ave,casual,2020,2020-04-18,2020-04-18,10:22:59,11:15:54,4,4,18,18


## 5. Misc. data cleaning

In [23]:
### Number of unique categories of bicycles to ride
new_df["rideable_type"].nunique()

3

In [26]:
### Convert rideable_type column and member_casual column to cateogry
# This should save some more memory

new_df["rideable_type"] = new_df["rideable_type"].astype("category")
new_df["member_casual"] = new_df["member_casual"].astype("category")

In [27]:
### 5.4gb is still high, but it's better than 7gb+
new_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14758113 entries, 0 to 14758321
Data columns (total 15 columns):
 #   Column              Dtype         
---  ------              -----         
 0   rideable_type       category      
 1   started_at          datetime64[ns]
 2   ended_at            datetime64[ns]
 3   start_station_name  object        
 4   end_station_name    object        
 5   member_casual       category      
 6   Year                int64         
 7   start_date          object        
 8   end_date            object        
 9   start_time          object        
 10  end_time            object        
 11  start_month         int64         
 12  end_month           int64         
 13  start_day           int64         
 14  end_day             int64         
dtypes: category(2), datetime64[ns](2), int64(5), object(6)
memory usage: 5.4 GB


In [28]:
#new_df.to_csv("bicycle_sharing_cleaned.csv",index=False)